<a href="https://colab.research.google.com/github/SimoneMonastero/Pink_Horizons/blob/main/CLASSIFIER/CV_noage_classifier_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CLASSIFICATION TASK (CV and no AGE)

In [191]:
import numpy as np #used a lot in signal processing
import pandas as pd #used with spreadshits
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression #read thedocumantation about it
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import recall_score

# Setting the seed of the random generator
SEED = 42
test_size = 0.10
n_folds = 5


In [192]:
CO = pd.read_excel('/content/DATI_COnoage.xlsx')
FA = pd.read_excel('/content/DATI_FALLnoage.xlsx')

print(CO)
print(FA)

       TUG   FSST  BERG  DGI  VAL
0    8.000   9.00    52   22    0
1    7.580   9.11    54   23    0
2   11.000  12.50    53   19    0
3    9.560  11.53    51   22    0
4   11.820  13.70    54   22    0
5    9.370  10.12    56   22    0
6    7.810   8.34    56   22    0
7   12.110  14.25    47   19    0
8   11.500  10.87    56   22    0
9    8.830  12.25    55   24    0
10   7.420  10.92    54   23    0
11   6.815   7.83    54   22    0
12   6.240   7.39    56   24    0
13   7.940  10.17    56   24    0
14   9.140   9.88    55   21    0
15   9.900  10.68    55   24    0
16   7.770   6.52    56   24    0
17  11.060  11.61    52   22    0
18  13.000  17.00    53   24    0
19  15.000  16.00    53   22    0
20  12.500  11.00    51   21    0
21  12.710  13.00    45   21    0
22   6.960  10.50    54   24    0
23   7.005  11.67    56   24    0
24  11.000  12.00    55   24    0
25   6.470  11.44    55   23    0
26  10.680  14.82    55   22    0
27  11.830  12.86    52   19    0
28  18.960  19

In [194]:
print("FA shape before NaN removal:", FA.shape)
print("CO shape before NaN removal:", CO.shape)

FA.dropna(axis=0, how='any', inplace=True)
CO.dropna(axis=0, how='any', inplace=True)

print("FA shape after NaN removal:", FA.shape)
print("CO shape after NaN removal:", CO.shape)
#


FA shape before NaN removal: (33, 5)
CO shape before NaN removal: (44, 5)
FA shape after NaN removal: (31, 5)
CO shape after NaN removal: (44, 5)


In [195]:
X = pd.concat([FA, CO], ignore_index=True)
display(X.head())

,TUG,FSST,BERG,DGI,VAL
0,8.56,10.32,55,23,1
1,11.01,14.55,48,18,1
2,11.05,14.60,51,22,1
3,18.18,20.52,50,19,1
4,11.05,18.30,54,23,1


In [196]:
# Re-concatenate FA and CO to restore X with all original columns
X = pd.concat([FA, CO], ignore_index=True)
print(X)

       TUG   FSST  BERG  DGI  VAL
0    8.560  10.32    55   23    1
1   11.010  14.55    48   18    1
2   11.050  14.60    51   22    1
3   18.180  20.52    50   19    1
4   11.050  18.30    54   23    1
..     ...    ...   ...  ...  ...
70  12.430  13.53    53   23    0
71  11.625  23.00    46   20    0
72  10.690  10.36    54   23    0
73   8.030  10.43    53   23    0
74  10.360   9.12    52   21    0

[75 rows x 5 columns]


In [197]:
# Now, X has all columns, so we can correctly select X_features
X_features = X.drop('VAL', axis=1)
display(X_features.head())

,TUG,FSST,BERG,DGI
0,8.56,10.32,55,23
1,11.01,14.55,48,18
2,11.05,14.60,51,22
3,18.18,20.52,50,19
4,11.05,18.30,54,23


In [198]:
Y = X['VAL']
print(Y)

0     1
1     1
2     1
3     1
4     1
     ..
70    0
71    0
72    0
73    0
74    0
Name: VAL, Length: 75, dtype: int64


CROSS VALIDATION


In [199]:
clf = LogisticRegression()

cv = KFold(n_splits=n_folds, shuffle=True, random_state=SEED)

for train_index, test_index in cv.split(X_features):
    print("Train:", train_index, " Test:", test_index)

Train: [ 1  2  3  5  6  7  8 11 13 14 15 16 17 19 20 21 22 23 24 25 26 27 29 30
 31 32 33 36 37 38 39 40 41 43 44 45 46 47 48 50 51 52 53 54 55 56 57 58
 59 60 62 65 66 67 68 70 71 72 73 74]  Test: [ 0  4  9 10 12 18 28 34 35 42 49 61 63 64 69]
Train: [ 0  1  2  3  4  6  8  9 10 11 12 13 14 15 17 18 19 20 21 23 24 25 26 27
 28 29 32 34 35 36 37 38 41 42 43 46 48 49 50 51 52 53 54 55 57 59 60 61
 62 63 64 65 67 68 69 70 71 72 73 74]  Test: [ 5  7 16 22 30 31 33 39 40 44 45 47 56 58 66]
Train: [ 0  1  2  4  5  7  9 10 11 12 14 15 16 18 20 21 22 23 24 26 27 28 29 30
 31 32 33 34 35 37 39 40 41 42 43 44 45 46 47 48 49 51 52 55 56 57 58 59
 60 61 63 64 65 66 67 68 69 70 71 73]  Test: [ 3  6  8 13 17 19 25 36 38 50 53 54 62 72 74]
Train: [ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 16 17 18 19 20 21 22 23 25 28
 29 30 31 33 34 35 36 37 38 39 40 42 44 45 47 49 50 51 52 53 54 56 58 59
 60 61 62 63 64 65 66 69 70 71 72 74]  Test: [11 15 24 26 27 32 41 43 46 48 55 57 67 68 73]
Train: [ 0  3  4  5 

In [200]:
score = cross_validate(clf, X=X_features, y=Y, cv=cv, return_train_score=True, return_estimator=True, scoring = 'roc_auc')
print("This is the score object:")
print (score)
print("Average AUC test set:", np.mean(score['test_score']))

This is the score object:
{'fit_time': array([0.01283026, 0.01226187, 0.00966668, 0.0102694 , 0.01231122]), 'score_time': array([0.00495386, 0.00387859, 0.0043087 , 0.00552869, 0.0054214 ]), 'estimator': [LogisticRegression(), LogisticRegression(), LogisticRegression(), LogisticRegression(), LogisticRegression()], 'test_score': array([0.73214286, 0.78      , 0.67857143, 0.5       , 0.48214286]), 'train_score': array([0.71759259, 0.67420814, 0.69444444, 0.73529412, 0.74652778])}
Average AUC test set: 0.6345714285714286


In [201]:
from sklearn.metrics import recall_score

sensitivities = []
specificities = []

for i, (train_index, test_index) in enumerate(cv.split(X_features, Y)):
    # Get the fitted estimator for the current fold
    fold_estimator = score['estimator'][i]

    # Get the test data for the current fold
    X_test_fold = X_features.iloc[test_index]
    y_test_fold = Y.iloc[test_index]

    # Make predictions on the test set for the current fold
    y_pred_fold = fold_estimator.predict(X_test_fold)

    # Calculate sensitivity (recall for class 1)
    sensitivity = recall_score(y_test_fold, y_pred_fold, pos_label=1)
    sensitivities.append(sensitivity)

    # Calculate specificity (recall for class 0)
    specificity = recall_score(y_test_fold, y_pred_fold, pos_label=0)
    specificities.append(specificity)

print("Average Sensitivity after cross-validation:", np.mean(sensitivities))
print("Average Specificity after cross-validation:", np.mean(specificities))

Average Sensitivity after cross-validation: 0.3771428571428571
Average Specificity after cross-validation: 0.8099999999999999
